In [1]:
%load_ext autoreload
%autoreload 2    

# Setup

In [9]:
# DB_NAME = "./memories.db"
# DB_NAME = "./storm.db"
# DB_NAME = "./hipporag.db"
DB_NAME = "./gpt_review.db"

In [10]:
from broai.experiments.vector_store import DuckVectorStore

In [11]:
from package.jargon_store import JargonStore, JargonRecord

In [12]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_29818/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [13]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_29818/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 174278.56it/s]


In [14]:
raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)
jargon_memory = JargonStore(db_name=DB_NAME, table="jargon_memory")

/tmp/ipykernel_29818/1440006260.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_29818/1440006260.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
/tmp/ipykernel_29818/1440006260.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)


# Test

In [15]:
from package.offline_flow import MemoryFlow, OfflineFlow

In [16]:
from agents.context_enricher import ContextEnricher
from agents.jargon_extractor import JargonExtractor

In [17]:
max_tokens = 500

memory_flow = MemoryFlow(raw_memory, enrich_memory, longterm_memory, jargon_memory)
offline_flow = OfflineFlow(
    knowledge_flow=memory_flow,
    context_enricher=ContextEnricher(),
    jargon_extractor=JargonExtractor(),
    max_tokens=max_tokens,
    overlap=int(max_tokens*0.3)
)

In [18]:
# source = "./docs/storm.md"
# source = "./docs/hipporag.md"
source = "./docs/gpt_review.pdf"
offline_flow.run(source=source, source_type="document")

/home/ec2-user/SageMaker/broai-researcher-assistant/package/offline_flow.py:57: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: pdf_to_markdown
  # with open(source, "r") as f:


Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 10/10 [00:00<00:00, 60.06it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]
/home/ec2-user/SageMaker/broai-researcher-assistant/package/offline_flow.py:65: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  consolidated_chunks = consolidate_markdown(chunks)
/home/ec2-user/SageMaker/broai-researcher-assistant/package/offline_flow.py:66: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  sections = get_markdown_sections(consolidated_chunks)
/home/ec2-user/SageMaker/broai-researcher-assistant/package/offline_flow.py:67: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  contexts = []
/home/ec2-user/SageMaker/broai-researcher-assistant/pa

Markdown headings: max(4)


  0%|          | 0/175 [00:00<?, ?it/s]/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the evidence of the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
100%|██████████| 175/175 [05:46<00:00,  1.98s/it]


0
